In [1]:
from requests import get
from time import time
import pandas as pd
from time import sleep
from random import randint
import numpy as np
import pickle
from bs4 import BeautifulSoup
from lxml import html
from re import sub
from decimal import Decimal
import string
import traceback
from time import sleep
from random import randint
from datetime import datetime
from dateutil.parser import parse



Scrape IMDB website for movies on keyword search remake.and the get the title,imdb_rating,runtimes,year,genres

In [4]:
url_list=[]
base_url = 'http://www.imdb.com'
all_movie_dict = []



for i in range(1,50):
    url_list.append('https://www.imdb.com/search/title?title_type=feature&release_date=2010-01-01,2013-12-31&languages=en&page={}&ref_=adv_nxt'.format(i))

    
def movie_data(container_soup,movie_url,remake='y'):
    movie = {}
    try:
        movie_response = get(movie_url)
        movie_soup = BeautifulSoup(movie_response.text)
    except:
        print("Error connecting to the individual movie link",movie_url)
        return
    title_obj = movie_soup.find("div", class_ ='title_wrapper')
    #transform_title = title.obj.h1.text
    #print(title.obj.h1.text.strip('('))
    title_details = container_soup.h3.a.text
    movie['title'] = title_details
    try:
        imdb_rating = movie_soup.find("div",class_ ='ratingValue')
        rating_value = imdb_rating.find("strong")
        movie['imdbRating'] = float(rating_value.span.text)
    except:
        movie['imdbRating'] = np.nan
    
    try:
        
        html_release = (movie_soup.findAll("h4", text="Release Date:"))[0].parent
        new_date =' '.join(((html_release.text).split())[2:6])
        d = new_date.split()
        new_d = d[1]+" "+d[0]+' '+d[2]
        dt = parse(new_d)
        movie['release_date'] = dt.strftime('%m/%d/%Y')
        dt_month=dt.strftime('%m')
        if int(dt_month) in [5,6,7,11,12]:
            movie['primetime']=1
        else:
            movie['primetime'] =0
    except: 
        movie['release_date'] = ''
        movie['primetime']=np.nan
    
    try:
        html_country = movie_soup.find(id="titleDetails")
        divs = html_country.findAll("div",class_="txt-block")
        country_value=''
        for div in divs:
            if div.h4.text == 'Country:':
                country_value = div.find("a").text
                movie['country'] = country_value
                break
    except:
        movie['country']=''
    try:
        html_language = movie_soup.find(id="titleDetails")
        divs = html_country.findAll("div",class_="txt-block")
        for div in divs:
            if div.h4.text == 'Language:':
                language_value = div.find("a").text
                movie['language'] = language_value
                break
    except:
        movie['language']=''
    
    try:
        divs = container_soup.find_all("p",class_="text-muted")
        rspans = divs[0].find_all("span")
        movie['certificate'] = rspans[0].text
        movie['runtime']= int(rspans[2].text.split()[0])
    except:
        
        movie['certificate'] =''
        movie['runtime']=np.nan
    
        
    #connections = movie_url + '/movieconnections/?tab=mc&ref_=tt_trv_cnn'
    
    try:
        genres_list = []
        genre_obj = movie_soup.find(id ='titleStoryLine')
        divs = genre_obj.findAll("div", class_ = 'see-more inline canwrap')
        for div in divs:
            if div.h4.text == 'Genres:':
                genres = div.find_all('a')
                for genre in genres:
                    genres_list.append(genre.text)
    except: 
        genres_list =[]
    movie['genre'] = genres_list
    
   
    try:
        budget_response = get(movie_url)
        tree = html.fromstring(budget_response.content)
        budget = tree.xpath('//*[@id="titleDetails"]/div[7]/text()')
        budget_value=''
        movie['budget']=np.nan
        for i in budget:
            if i.find('$') != -1:
                budget_value = i
                movie['budget'] = int(Decimal(sub(r'[^\d.]','',budget_value)))
                break
            else:
                movie['budget']=np.nan
    except:
        
        movie['budget'] =np.nan
    try:
        gross_value=''
        movie['gross'] = np.nan
        gross = tree.xpath('//*[@id="titleDetails"]/div[9]/text()')
        for i in gross: 
            if i.find('$') != -1:
                gross_value = i
                movie['gross'] = int(Decimal(sub(r'[^\d.]','',gross_value)))
                break
            else:
                movie['gross']=np.nan
        
    except:
        movie['gross'] = np.nan
    try:    
        cg_value=''
        movie['cgross'] = np.nan
        c_gross = tree.xpath('//*[@id="titleDetails"]/div[10]/text()')
        for i in c_gross:
            if i.find('$') != -1:
                cg_value = i
                
                movie['cgross'] = int(Decimal(sub(r'[^\d.]','',cg_value)))
                break
            else:
                movie['cgross']=np.nan
    except:
        movie['cgross']= np.nan
    print(movie)
    return movie

def scrape_imdb_list(list_url):
    try:
        response = get(list_url)
        page_content = BeautifulSoup(response.text)
        print("connecting to url",list_url)
        print("Sleeping before next page request ")
        sleep(randint(8,15))
        #num_requests+= 1
        #elapsed_time = time() - start_time
        #print('Request:{}; Frequency: {} requests/s'.format(num_requests, num_requests/elapsed_time))
        #clear_output(wait = True)

    except Exception as ex:
        print("Error connecting to url in scrape_imdb_list", list_url)
        traceback.print_exc()
        print("Exception",ex)
    
    mv_containers = page_content.find_all(class_='lister-item-content')
    dict = []
    for container in mv_containers:
        #container=mv_containers
        url = base_url +container.h3.a.get('href')
        ref = url.rfind('/')
        clean_url = url[:ref]
        try:
            dict.append(movie_data(container,clean_url))
        except Exception as ex:
            traceback.print_exc()
            print("Exception",ex)
            pass
    print(dict)
    return dict

for url in url_list:
    
    try:
        all_movie_dict = all_movie_dict+scrape_imdb_list(url)
    except:
        pass
with open('imdb_allmovies_new2.pkl', 'wb') as picklefile:
        pickle.dump(all_movie_dict, picklefile)


connecting to url https://www.imdb.com/search/title?title_type=feature&release_date=2010-01-01,2013-12-31&languages=en&page=1&ref_=adv_nxt
Sleeping before next page request 
{'title': 'Scott Pilgrim vs. the World', 'imdbRating': 7.5, 'release_date': '08/13/2010', 'primetime': 0, 'country': 'USA', 'language': 'English', 'certificate': 'PG-13', 'runtime': 112, 'genre': [' Action', ' Comedy', ' Fantasy', ' Romance'], 'budget': 60000000, 'gross': 31494270, 'cgross': 132000000}
{'title': 'Inception', 'imdbRating': 8.8, 'release_date': '07/16/2010', 'primetime': 1, 'country': 'USA', 'language': 'English', 'certificate': 'PG-13', 'runtime': 148, 'genre': [' Action', ' Adventure', ' Sci-Fi', ' Thriller'], 'budget': 160000000, 'gross': 292576195, 'cgross': 825532764}
{'title': 'The Conjuring', 'imdbRating': 7.5, 'release_date': '07/19/2013', 'primetime': 1, 'country': 'USA', 'language': 'English', 'certificate': 'R', 'runtime': 112, 'genre': [' Horror', ' Mystery', ' Thriller'], 'budget': 20000

{'title': 'The Girl with the Dragon Tattoo', 'imdbRating': 7.8, 'release_date': '12/21/2011', 'primetime': 1, 'country': 'USA', 'language': 'English', 'certificate': 'R', 'runtime': 158, 'genre': [' Crime', ' Drama', ' Mystery', ' Thriller'], 'budget': 90000000, 'gross': 102515793, 'cgross': 232617430}
{'title': 'The Amazing Spider-Man', 'imdbRating': 7.0, 'release_date': '07/03/2012', 'primetime': 1, 'country': 'USA', 'language': 'English', 'certificate': 'PG-13', 'runtime': 136, 'genre': [' Action', ' Adventure', ' Sci-Fi'], 'budget': 230000000, 'gross': 262030663, 'cgross': 757930663}
{'title': 'Warrior', 'imdbRating': 8.2, 'release_date': '09/09/2011', 'primetime': 0, 'country': 'USA', 'language': 'English', 'certificate': 'PG-13', 'runtime': 140, 'genre': [' Drama', ' Sport'], 'budget': 25000000, 'gross': 13657115, 'cgross': 23057115}
{'title': 'Man of Steel', 'imdbRating': 7.1, 'release_date': '06/14/2013', 'primetime': 1, 'country': 'USA', 'language': 'English', 'certificate': '

connecting to url https://www.imdb.com/search/title?title_type=feature&release_date=2010-01-01,2013-12-31&languages=en&page=2&ref_=adv_nxt
Sleeping before next page request 
{'title': 'The Perks of Being a Wallflower', 'imdbRating': 8.0, 'release_date': '10/12/2012', 'primetime': 0, 'country': 'USA', 'language': 'English', 'certificate': 'PG-13', 'runtime': 102, 'genre': [' Drama', ' Romance'], 'budget': 13000000, 'gross': 17738570, 'cgross': nan}
{'title': 'Iron Man 2', 'imdbRating': 7.0, 'release_date': '05/07/2010', 'primetime': 1, 'country': 'USA', 'language': 'English', 'certificate': 'PG-13', 'runtime': 124, 'genre': [' Action', ' Adventure', ' Sci-Fi'], 'budget': 200000000, 'gross': 312433331, 'cgross': 623933331}
{'title': 'Beautiful Creatures', 'imdbRating': 6.2, 'release_date': '02/14/2013', 'primetime': 0, 'country': 'USA', 'language': 'English', 'certificate': 'PG-13', 'runtime': 124, 'genre': [' Drama', ' Fantasy', ' Romance'], 'budget': 60000000, 'gross': 19452138, 'cgros

{'title': 'Maniac', 'imdbRating': 6.1, 'release_date': '01/02/2013', 'primetime': 0, 'country': 'France', 'language': 'English', 'certificate': 'Not Rated', 'runtime': 89, 'genre': [' Horror', ' Thriller'], 'budget': 5571, 'gross': nan, 'cgross': nan}
{'title': 'Blue Valentine', 'imdbRating': 7.4, 'release_date': '01/28/2011', 'primetime': 0, 'country': 'USA', 'language': 'English', 'certificate': 'R', 'runtime': 112, 'genre': [' Drama', ' Romance'], 'budget': 3500000, 'gross': 9737892, 'cgross': 16566240}
{'title': 'The Impossible', 'imdbRating': 7.6, 'release_date': '01/04/2013', 'primetime': 0, 'country': 'Spain', 'language': 'English', 'certificate': 'PG-13', 'runtime': 114, 'genre': [' Drama', ' History', ' Thriller'], 'budget': 45000000, 'gross': 143818, 'cgross': 19019882}
{'title': 'Grown Ups 2', 'imdbRating': 5.4, 'release_date': '07/12/2013', 'primetime': 1, 'country': 'USA', 'language': 'English', 'certificate': 'PG-13', 'runtime': 101, 'genre': [' Comedy'], 'budget': 800000

connecting to url https://www.imdb.com/search/title?title_type=feature&release_date=2010-01-01,2013-12-31&languages=en&page=3&ref_=adv_nxt
Sleeping before next page request 
{'title': 'Moneyball', 'imdbRating': 7.6, 'release_date': '09/23/2011', 'primetime': 0, 'country': 'USA', 'language': 'English', 'certificate': 'PG-13', 'runtime': 133, 'genre': [' Biography', ' Drama', ' Sport'], 'budget': 50000000, 'gross': 75605492, 'cgross': 110206216}
{'title': 'Moonrise Kingdom', 'imdbRating': 7.8, 'release_date': '06/29/2012', 'primetime': 1, 'country': 'USA', 'language': 'English', 'certificate': 'PG-13', 'runtime': 94, 'genre': [' Adventure', ' Comedy', ' Drama', ' Romance'], 'budget': 16000000, 'gross': 45512466, 'cgross': 68263166}
{'title': 'ParaNorman', 'imdbRating': 7.0, 'release_date': '08/17/2012', 'primetime': 0, 'country': 'USA', 'language': 'English', 'certificate': 'PG', 'runtime': 92, 'genre': [' Animation', ' Adventure', ' Comedy', ' Family', ' Fantasy'], 'budget': 14087050, '

...

Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/urllib3/connection.py", line 171, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/anaconda3/lib/python3.6/site-packages/urllib3/util/connection.py", line 56, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/anaconda3/lib/python3.6/socket.py", line 745, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 8] nodename nor servname provided, or not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 343, in _make_request
    self._validate_conn(conn)
  File "/anaconda3/lib/python3.6/site-packages/urllib3/conn

Error connecting to url in scrape_imdb_list https://www.imdb.com/search/title?title_type=feature&release_date=2010-01-01,2013-12-31&languages=en&page=48&ref_=adv_nxt
Exception HTTPSConnectionPool(host='www.imdb.com', port=443): Max retries exceeded with url: /search/title?title_type=feature&release_date=2010-01-01,2013-12-31&languages=en&page=48&ref_=adv_nxt (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x120261550>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))


Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/urllib3/connection.py", line 171, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/anaconda3/lib/python3.6/site-packages/urllib3/util/connection.py", line 56, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/anaconda3/lib/python3.6/socket.py", line 745, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 8] nodename nor servname provided, or not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 343, in _make_request
    self._validate_conn(conn)
  File "/anaconda3/lib/python3.6/site-packages/urllib3/conn

Error connecting to url in scrape_imdb_list https://www.imdb.com/search/title?title_type=feature&release_date=2010-01-01,2013-12-31&languages=en&page=49&ref_=adv_nxt
Exception HTTPSConnectionPool(host='www.imdb.com', port=443): Max retries exceeded with url: /search/title?title_type=feature&release_date=2010-01-01,2013-12-31&languages=en&page=49&ref_=adv_nxt (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x12062e080>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))


Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/urllib3/connection.py", line 171, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/anaconda3/lib/python3.6/site-packages/urllib3/util/connection.py", line 56, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/anaconda3/lib/python3.6/socket.py", line 745, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 8] nodename nor servname provided, or not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 343, in _make_request
    self._validate_conn(conn)
  File "/anaconda3/lib/python3.6/site-packages/urllib3/conn